In [52]:
import matplotlib.pyplot as plt
import numpy as np
use_notebook = True

In [53]:
# Mean squared error from residual
def mse_from_e(residual):
    return sum(np.square(residual))/len(residual)

# Mean Squared Error of X.w - y
def mean_squared_error(X, y, w):
    residual = np.dot(X,w) - y
    return sum(np.square(residual))/len(residual)


In [54]:
# Generate a sawtooth weight vector
def genu(d, m):
    u = np.arange(d) * (2 * (np.arange(d) % 2) - 1)
    u = m * u / np.sqrt(np.dot(u, u))
    return u

# Generate random data(0,1) of size n X d.
def genx(n, d):
    X = np.random.randint(0, 2, (n, d))
    X[:,int(d/2)] = 1
    return X

# Generate targets and add noise
def gent(X, u, noise):
    n = X.shape[0]
    y = np.dot(X, u).reshape(n, 1)
    y += noise * np.var(y) * np.random.randn(n, 1)
    return y

# Generate data, weights, and targets
def gimme_data_regres(n, d, noise=0.1):
    u = genu(d, 1.0)
    X = genx(n, d)
    y = gent(X, u, noise)
    mse_gen = mean_squared_error(X, y, u)
    print('Generator Loss={0:8.5f}\n'.format(mse_gen))
    return X, u, y

In [55]:
# Plot loss as a function of epoch
def loss_plotter(vlist, fname):
    vr = vlist[0]
    vn = vlist[1]
    if not use_notebook:
        plt.figure(fname)
        plt.plot(range(1, 1+len(vr)), vr,
           range(1, 1+len(vn)), vn,
           linewidth=2, linestyle='-', marker='o')
    plt.legend(('rep', 'nor'))
    plt.grid()
    xt = np.arange(1, 1 + max(len(vr), len(vn)))
    _ = plt.xticks(xt)
    _ = plt.xlabel('Epoch', fontsize=14)
    _ = plt.ylabel(fname, fontsize=14)
    
    if not use_notebook:
        plt.show(block=False)
    return


# Scatter plot of predicted vs. observed targets
def loss_scatter(X, y, w, fname):
    if not use_notebook:
        plt.figure(fname)
    plt.scatter(y, X.dot(w), edgecolors=(0,0,0))
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    plt.grid()
    plt.xlabel('$y$', fontsize=14)
    plt.ylabel('$\hat{y}$', fontsize=14)
    if not use_notebook:
        plt.show(block=False)

In [56]:
# sample new index (w/ or w/o replacement)
def sample_new_index(d, replace=1):
    if replace:
        ## ACT3
        ind = np.random.randint(0, d)
    else:
        if 'prm' not in sample_new_index.__dict__:
            sample_new_index.prm = np.random.permutation(d)
            sample_new_index.head = 0
            
        ## ACT4
        ind = sample_new_index.prm[sample_new_index.head]
        sample_new_index.head += 1
        
        if sample_new_index.head == d:
            sample_new_index.head = 0
            del sample_new_index.prm
    return ind

In [57]:
# calculate the change to w[j] wrt current margins z
# xjs is the squared norm of the jth column of X, a.k.a. ||xj||^2
def delta_wj(e, xj, xjs):
    ## ACT5
    delta = e * xj / xjs
    return delta

# Return new values for w[j] and residual
def update(wj, e, xj, xjs):
    ## ACT6
    wj_new = wj + e * xj / xjs
    return wj_new, e - e * xj * xj / xjs

In [58]:
# Initialize all variables using the zero vector for w
# (Initialize w as the zero vector)
# You should return w, xjs, residual
def initialize(X, y):
    n, d = X.shape
    w = np.zeros(d)  # Initialize w as the zero vector
    xjs = np.sum(X ** 2, axis=0)  # Compute squared norms of columns of X
    residual = y.copy()  # Initialize residual as y
    return w, xjs, residual

In [59]:
# Check whether termination condition is met
def mse_check(mse_p, mse_c, eps):
    ## ACT8
    return abs(mse_p - mse_c) <= eps

In [60]:
# Linear regression using coordinate decent
def linear_regression_cd(X, y, epochs=100, eps=0.001, replace=1):
    w, xjs, residual = initialize(X, y)
    mse_cd = [mse_from_e(residual)]
    n, d = X.shape
    for e in range(d * epochs):
        j = sample_new_index(d, replace)
        xj = X[:,j].reshape(n, 1)
        w[j], residual = update(w[j], residual, xj, xjs[j])
        if (e + 1) % d == 0:
            mse_cd.append(mse_from_e(residual))
            print('Epoch: {0:2d}  MSE: {1:5.3f}'.format(int((e+1)/d), mse_cd[-1]))
            if mse_check(mse_cd[-2], mse_cd[-1], eps): break
    return w, mse_cd

In [ ]:
# ---------------- Main for linear regression using Coordinate Descent --------------

np.random.seed(17)
n, d, noise = 1000, 20, 1.0
myeps = 1e-4

[X, u, y] = gimme_data_regres(n, d, noise)

mse_list = []
[wr, mse_r] = linear_regression_cd(X, y, eps=myeps)
mse_list.append(mse_r)
[wn, mse_n] = linear_regression_cd(X, y, eps=myeps, replace=0)
mse_list.append(mse_n)

if not use_notebook:
    plt.close('all')
loss_plotter(mse_list, 'MSE')
loss_scatter(X, y, wn, 'True vs. Predicted Outcome')